In [4]:
import pandas as pd


In [5]:
df=pd.read_csv("data.csv")
df=df.drop(["etest_p","mba_p"],axis=1)

In [6]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

object_cols=['gender','workex','specialisation','status']

Q1 = df['hsc_p'].quantile(0.25)
Q3 = df['hsc_p'].quantile(0.75)
IQR = Q3 - Q1 
filter = (df['hsc_p'] >= Q1 - 1.5 * IQR) & (df['hsc_p'] <= Q3 + 1.5 *IQR)
placement_filtered=df.loc[filter]

label_encoder = LabelEncoder()
for col in object_cols:
    placement_filtered[col] = label_encoder.fit_transform(placement_filtered[col])
placement_filtered.head()

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,specialisation,status,salary
0,1,1,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,0,1,1,270000.0
1,2,1,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,1,0,1,200000.0
2,3,1,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,0,0,1,250000.0
3,4,1,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,0,1,0,NaN
4,5,1,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,0,0,1,425000.0


In [11]:
dummy_hsc_b=pd.get_dummies(placement_filtered['hsc_b'], prefix='dummy')
dummy_ssc_b=pd.get_dummies(placement_filtered['ssc_b'], prefix='dummy')
dummy_hsc_s=pd.get_dummies(placement_filtered['hsc_s'], prefix='dummy')
dummy_degree_t=pd.get_dummies(placement_filtered['degree_t'], prefix='dummy')
placement_coded = pd.concat([placement_filtered,dummy_hsc_s,dummy_degree_t,dummy_hsc_b,dummy_ssc_b],axis=1)
placement_coded=placement_coded[placement_coded['status']==1]
placement_coded.drop(['hsc_b','degree_t','ssc_b','hsc_s','status'],axis=1, inplace=True)
placement_coded


,sl_no,gender,ssc_p,hsc_p,degree_p,workex,specialisation,salary,dummy_Arts,dummy_Commerce,dummy_Science,dummy_Comm&Mgmt,dummy_Others,dummy_Sci&Tech,dummy_Central,dummy_Others,dummy_Central,dummy_Others
0,1,1,67.00,91.00,58.00,0,1,270000.0,0,1,0,0,0,1,0,1,0,1
1,2,1,79.33,78.33,77.48,1,0,200000.0,0,0,1,0,0,1,0,1,1,0
2,3,1,65.00,68.00,64.00,0,0,250000.0,1,0,0,1,0,0,1,0,1,0
4,5,1,85.80,73.60,73.30,0,0,425000.0,0,1,0,1,0,0,1,0,1,0
7,8,1,82.00,64.00,66.00,1,0,252000.0,0,0,1,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,210,1,62.00,72.00,65.00,0,0,216000.0,0,1,0,1,0,0,1,0,1,0
210,211,1,80.60,82.00,77.60,0,0,400000.0,0,1,0,1,0,0,0,1,0,1
211,212,1,58.00,60.00,72.00,0,0,275000.0,0,0,1,0,0,1,0,1,0,1
212,213,1,67.00,67.00,73.00,1,0,295000.0,0,1,0,1,0,0,0,1,0,1


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


X=placement_coded.drop(['salary'],axis=1)
y=placement_coded['salary']

X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=1)
X_train


,sl_no,gender,ssc_p,hsc_p,degree_p,workex,specialisation,dummy_Arts,dummy_Commerce,dummy_Science,dummy_Comm&Mgmt,dummy_Others,dummy_Sci&Tech,dummy_Central,dummy_Others,dummy_Central,dummy_Others
69,70,1,73.0,73.0,66.0,1,0,0,0,1,0,0,1,1,0,1,0
44,45,0,77.0,73.0,81.0,1,0,0,1,0,1,0,0,0,1,0,1
153,154,1,49.0,59.0,65.0,1,0,0,0,1,0,0,1,0,1,0,1
146,147,1,62.0,63.0,66.0,0,1,0,0,1,1,0,0,0,1,1,0
138,139,0,82.0,64.0,73.0,1,0,0,0,1,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,200,1,69.0,65.0,57.0,0,1,0,1,0,1,0,0,0,1,0,1
204,205,0,74.0,73.0,73.0,1,0,0,1,0,1,0,0,0,1,0,1
107,108,1,82.0,90.0,83.0,0,1,0,1,0,1,0,0,0,1,0,1
209,210,1,62.0,72.0,65.0,0,0,0,1,0,1,0,0,1,0,1,0


In [14]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

-0.5470746626312462